# Comparison of GC of cephalopods in Ensembl or NCBI releases
# batch 3 - with spaces and "N" in genome with different color than cmap

<font color='red'>**Guide of usage**</font>

<font color='red'>**New version for Cephalopods**</font>

all done in batch - data are prepared by **batch_for_cephalophod_GC_repeats_profile_PLOS.py** in the same directory

Marta Vohnoutová, Radka Symonová

version 0.3

Changes:
    
    - dots in graphs - same density - adjasted to left
    - pick up chromosomes only - universal
    - with spaces and "N" in genome with different color than cmap

## <font color='red'>2.Import section</font>

In [47]:
#import pyensembl

from Bio import SeqIO
from Bio.SeqUtils import gc_fraction
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
from matplotlib.ticker import MaxNLocator
from matplotlib import font_manager, cm
plt.rcParams["figure.figsize"] = [20.00, 20.00]
import pandas as pd
import numpy as np
import os
import csv
from collections import OrderedDict
from PIL import Image
from glob import glob
import pickle

## <font color='red'>3.Configuration and download section</font>

In [48]:
!pwd

/mnt/data/marta/evan/PlosComp/dataOIST


# save what_to_operate for next usage

In [49]:
# read what_to_operate from pickle

file = open('/home/marta/marta/marta/evan/PlosComp/dataOIST/pickles/what_to_operate_Prochlorococcus_marinus.pkl','rb')
what_to_operate = pickle.load(file)
file.close()

In [50]:
what_to_operate

[['/home/marta/marta/marta/evan/PlosComp/dataOIST/Ansgar/Prochlorococcus_marinus/GCF_000015665.1_ASM1566v1_genomic.fna',
  'Prochlorococcus_marinus',
  [100],
  'rec.id.startswith("NC")',
  'Ansgar']]

### 3.1 Variable part - replace with right values

In [51]:
where_i_am = '/mnt/data/marta/evan/PlosComp/dataOIST/' # replace with your path
column_name = 'GC DNA' # replace with your column name

#### Create subdirectory for particular species

## Merge all graph images 

#### Prepare small graphs without colorbars

In [52]:
def how_many(my_seq,string):
    my_sum=0
    for base in string:
        my_sum += my_seq.count(base)
    return my_sum

In [53]:
# Constants for graph
column_name = 'GC DNA' # replace with your column name
# Set a special value that should be colored differently
special_value = 0.0

special_color = (0.0,0.0,0.999,0.999)

bottom = cm.get_cmap('RdYlGn', 255)
special_color = (0.0,0.0,0.999,0.999)
newcolors = np.vstack((special_color,bottom(np.linspace(0, 1, 255))))
new_cmap = ListedColormap(newcolors, name='new_RdYlGn')

slight_move=0.009         

In [54]:
# adjusted chromosome density
for my_fasta_file in what_to_operate:  #highest level
    for my_window in my_fasta_file[2]:
        print(my_fasta_file[0],my_window)
        sec_values = '' # replace with your release number
        c=[]

        GC_chromosoms=OrderedDict()
        chomosome_no=0

        gc_values=tuple()

        d_sec_values = dict()

        max_len=max([len(rec.seq) for rec in SeqIO.parse(f"{my_fasta_file[0]}", "fasta")]) # max len to adjust x axis
       
        for rec in SeqIO.parse(f"{my_fasta_file[0]}", "fasta"): 
            GC_chromosoms_windowed={}  # per chromosome
            if eval(my_fasta_file[3]):  # chromosomes only
                # count GC values via sliding windows
                c.append(rec.id)
                sec_values += rec.seq
                if len(sec_values) >= 10*my_window:
                    gc_values = gc_values + tuple(gc_fraction(sec_values[i:i+my_window]) for i in range(0,len(sec_values),my_window)) # replace with your release number
                    sec_values = ''
        
                # chromosomes
                soft_mask= how_many(rec.seq,'acgt')
                ALL_all=how_many(rec.seq,'acgtACGT')
                GC_fraction=gc_fraction(rec.seq)
                chomosome_no += 1
                
                try:
                    GC_chromosoms[str(chomosome_no) + '+' + rec.id]=((GC_fraction,soft_mask/ALL_all),len(rec.seq))  # we distract N and n   
                except ZeroDivisionError:
                    GC_chromosoms[str(chomosome_no)+'+'+ rec.id]=((GC_fraction,0.0), len(rec.seq)) # distract N and n
                  
        
                # count bases
                for base in 'acgtACGTNn':
                    if base in d_sec_values:
                        d_sec_values[base] += rec.seq.count(base)
                    else:
                        d_sec_values[base] = rec.seq.count(base)

                               
                #print(rec.id)  
                for w in range(0,len(rec.seq),my_window):
              
                    #print(range(i,i+window),len(gc_values_94[i:i+window]))
                    soft_mask=sum(1 for i in rec.seq[w:w+my_window] if i in 'acgt')
                    ALL_all=sum(1 for i in rec.seq[w:w+my_window] if i in 'acgtACGT')
                    GC_fraction=gc_fraction(rec.seq[w:w+my_window])
            
                    try:
                        GC_chromosoms_windowed[str(w)+'|'+ rec.id]=(GC_fraction,soft_mask/ALL_all)  # distract N and n    
                        #print(range(i,i+window),len(gc_values_94[i:i+window])) 
                    except ZeroDivisionError:
                        GC_chromosoms_windowed[str(w)+'|'+ rec.id]=(GC_fraction,0.0)  # distract N and n
            
                #fig = plt.figure(figsize=(int(50* (len(rec.seq)/max_len)),int(20* (len(rec.seq)/max_len))))
                fig = plt.figure(figsize=(25,10))
                ax1 = fig.add_subplot(111)
        
                names = list(GC_chromosoms_windowed.keys())
                v = list(GC_chromosoms_windowed.values())
                v1=[100 * i[0] for i in v]
                #v2=[i[1] for i in v]
                names_part=[n.split('|')[0] for n in names]
                values = np.array([special_value if i[0]+i[1] == 0 else i[1]+slight_move if (i[1] ==0 and i[0] != 0) else i[1] for i in v])  # values for special_indices
       
                sc=ax1.scatter(names_part,v1, s=5, c=values, cmap=new_cmap, marker="o", label='Blue dots are "NNnn" or spaces' ) # only GC fraction in y axe, only chromosomes
                #plt.xlim(0, max_len)
                #plt.gca().xaxis.set_major_locator(MaxNLocator(nbins=6))
                
                #locator=MaxNLocator(prune='both',nbins=10) # Adjust the number of ticks as needed
                #ax1.yaxis.set_major_locator(locator)
                ax1.xaxis.set_ticks(np.arange(0, max_len//my_window, max_len//12000))   
                #ax1.set_xlim(0, max_len)
                #plt.xlim(0, max_len)
                
                plt.title(f'GC% values of {my_fasta_file[1]} - {my_window} chromosome {rec.id}', fontsize = 30) # replace the animal name and release no.  
                plt.xticks(fontsize = 20, rotation = 30)
                plt.yticks(fontsize = 20)
                plt.legend(labelcolor='blue')
                
                plt.savefig(f'{where_i_am}{my_fasta_file[4]}/{my_fasta_file[1]}/{my_fasta_file[1]}_{my_window}_nobar_soft_unmask_{rec.id}.{len(rec.seq)}.png')  # replace the animal name and release no.
        
                #plt.show()
                plt.close('all')
        print(f'Done {my_fasta_file[1]} {my_window}')

/home/marta/marta/marta/evan/PlosComp/dataOIST/Ansgar/Prochlorococcus_marinus/GCF_000015665.1_ASM1566v1_genomic.fna 100
Done Prochlorococcus_marinus 100


#### Read graphs one by one and merge them all

In [55]:
# list of all graph images sorted to chromosomes no.

In [56]:
def get_concat(all_images,im_width,im_height):
    
    images = [Image.open(all_images[i]) for i in range(len(all_images))]
    if len(all_images)%2 == 0:
        how_high=len(all_images)
    else:
        how_high=len(all_images)+1
        
    dst = Image.new('RGB', (2*im_width, int((how_high * im_height)/2)))
    dst.paste(Image.open(f'{directories_only(my_fasta_file[0])}headtitle_{my_fasta_file[1]}_{my_window}.png'), (0, 0))
        
    for i in range(0,len(images)+1,2):
        try:
            dst.paste(images[i], (0, int((i * im_height)/2)))
        except IndexError:
            pass
        try:            
            dst.paste(images[i+1], (im_width,  int((i * im_height)/2)))
        except IndexError:
            pass
    return dst

In [57]:
my_pwd=!pwd
for my_fasta_file in what_to_operate:  #highest level
    for my_window in my_fasta_file[2]:
        #print(directories_only(my_fasta_file[0]),my_window)
        # cycle for all nobar images for species and window   
        
        dir_name= f'{my_pwd[0]}' + '/'+f'{what_to_operate[0][4]}' + '/'+ f'{what_to_operate[0][1]}'+'/'
        file_name= dir_name + '*' + f'{my_window}' +'_nobar*.png'
        all_images = sorted([f for f in glob(f"{file_name}")],key=lambda x: os.path.getsize(x), reverse = True)
        #print(all_images)
        images = [Image.open(f"{x}") for x in all_images]
        widths, heights = zip(*(i.size for i in images))
        max_width = max(widths)
        total_height = sum(heights)
        new_im = Image.new('RGB', (max_width, total_height))

        y_offset = 0
        for im in images:
            new_im.paste(im, (0,y_offset,))
            y_offset += im.size[1]

        new_im.save(f'{dir_name}{my_fasta_file[1]}_{my_window}_all_graphs.png')
        
        # Read small graphs one by one and merge them all to two columns¶
        fig = plt.figure(figsize=(0.01,0.01))
        plt.title(f'{my_fasta_file[1]} - sliding window {my_window} ', fontsize = 18)  # Replace the animal name

        plt.savefig(f'{dir_name}{my_fasta_file[1]}_{my_window}.png')  # Replace the animal name
        #plt.show()
        
        plt.close('all')
        
        im = Image.open(all_images[0])
        im_width=im.size[0]
        im_height=im.size[1]
        im.close()
        
        get_concat(all_images,im_width,im_height).save(f'{dir_name}{my_fasta_file[1]}_{my_window}.png')
        print(f'{my_window}_{my_fasta_file[1]}')

FileNotFoundError: [Errno 2] No such file or directory: '/home/marta/marta/marta/evan/PlosComp/dataOIST/Ansgar/Prochlorococcus_marinus/headtitle_Prochlorococcus_marinus_100.png'

In [42]:
dir_name

'/mnt/data/marta/evan/PlosComp/dataOIST/Fish/Sander/*1000_nobar*.png'

# Put all png images to pdf file

In the end all the files are put together into the pdf file.